In [4]:
import os
from PIL import Image

# --- Define the base directory where 'tb' category images are stored ---
image_dir = r"C:\Users\muham\Downloads\archive (1)\tbx11k-simplified\yolo_training\data\tb"
valid_exts = {'.jpg', '.jpeg', '.png', '.bmp', '.tiff'}

image_data = []

if not os.path.isdir(image_dir):
    print(f"❌ Error: Image directory not found at {image_dir}")
else:
    for file in os.listdir(image_dir):
        ext = os.path.splitext(file)[1].lower()
        if ext in valid_exts:
            full_path = os.path.join(image_dir, file)
            image_data.append({
                "filename": file,
                "path": full_path
            })

    print(f"\n✅ Loaded {len(image_data)} training images from 'tb' category.")

    # --- Optional: Show first image as verification ---
    if image_data:
        try:
            img = Image.open(image_data[0]['path'])
            print(f"\n🖼️ Preview: {image_data[0]['filename']} ({img.format}, {img.size})")
            img.show()
        except Exception as e:
            print(f"❌ Could not load image: {e}")
    else:
        print("⚠️ No images were found with valid extensions.")




✅ Loaded 800 training images from 'tb' category.

🖼️ Preview: tb0003.png (PNG, (512, 512))


In [14]:
import os
import pandas as pd
from shutil import copyfile, rmtree # Added rmtree for cleanup
from sklearn.model_selection import train_test_split
import ast # For safely evaluating string-represented dictionaries

# === 1. Configuration ===
# Base directory where the YOLO dataset structure will be created
base_dir = r"D:\yolo_training_tb_only" # Changed to a new clean directory
# Path to your original CSV file
csv_path = r"D:\yolo_training\data.csv" # Assuming this is where your original data.csv is
# Directory where your source images are located
image_source_dir = r"D:\yolo_training\data\tb" # Source of all .png images

# Define output directories
images_out_dir = os.path.join(base_dir, "images")
labels_out_dir = os.path.join(base_dir, "labels")
train_images_dir = os.path.join(images_out_dir, "train")
val_images_dir = os.path.join(images_out_dir, "val")
train_labels_dir = os.path.join(labels_out_dir, "train")
val_labels_dir = os.path.join(labels_out_dir, "val")

# YOLO class configuration
class_name = "tb"
class_id = 0 # Since we only have one class 'tb'

# Train/Validation split ratio
VAL_SPLIT_RATIO = 0.2
RANDOM_STATE = 42

# === 2. Cleanup and Directory Creation ===
# Clean up old directories if they exist to ensure a fresh start
if os.path.exists(base_dir):
    print(f"🧹 Cleaning up old directory: {base_dir}")
    rmtree(base_dir)

# Create the required YOLO directory structure
os.makedirs(train_images_dir, exist_ok=True)
os.makedirs(val_images_dir, exist_ok=True)
os.makedirs(train_labels_dir, exist_ok=True)
os.makedirs(val_labels_dir, exist_ok=True)
print(f"📁 Created YOLO directory structure in: {base_dir}")

# === 3. Load and Filter Data ===
df_all = pd.read_csv(csv_path)
print(f"📄 Loaded {len(df_all)} rows from {csv_path}")

# Filter out rows where 'bbox' is 'none'
df_bbox = df_all[df_all['bbox'] != 'none'].copy()
print(f"👍 Filtered to {len(df_bbox)} rows with bounding boxes.")

if df_bbox.empty:
    print("❌ No rows with bounding boxes found. Exiting.")
    exit()

# === 4. Bounding Box Conversion and Class Assignment ===
# Helper function to convert bbox string/dict to YOLO format
def bbox_to_yolo_format(bbox_item, img_width, img_height):
    """
    Converts a bounding box from {'xmin', 'ymin', 'width', 'height'}
    or {'xmin', 'ymin', 'xmax', 'ymax'} format to YOLO format.
    (x_center_norm, y_center_norm, width_norm, height_norm)
    """
    if isinstance(bbox_item, str):
        try:
            bbox = ast.literal_eval(bbox_item)
        except (ValueError, SyntaxError) as e:
            print(f"Error parsing bbox string: {bbox_item}. Error: {e}")
            return None
    elif isinstance(bbox_item, dict):
        bbox = bbox_item
    else:
        print(f"Invalid bbox_item type: {type(bbox_item)}")
        return None

    img_w = float(img_width)
    img_h = float(img_height)

    # Check if bbox is in (xmin, ymin, width, height) format
    if 'width' in bbox and 'height' in bbox:
        xmin = float(bbox['xmin'])
        ymin = float(bbox['ymin'])
        bbox_w = float(bbox['width'])
        bbox_h = float(bbox['height'])
        
        x_center = (xmin + bbox_w / 2) / img_w
        y_center = (ymin + bbox_h / 2) / img_h
        norm_w = bbox_w / img_w
        norm_h = bbox_h / img_h
    # Check if bbox is in (xmin, ymin, xmax, ymax) format
    elif 'xmax' in bbox and 'ymax' in bbox:
        xmin = float(bbox['xmin'])
        ymin = float(bbox['ymin'])
        xmax = float(bbox['xmax'])
        ymax = float(bbox['ymax'])

        x_center = ((xmin + xmax) / 2) / img_w
        y_center = ((ymin + ymax) / 2) / img_h
        norm_w = (xmax - xmin) / img_w
        norm_h = (ymax - ymin) / img_h
    else:
        print(f"Unsupported bbox format: {bbox}")
        return None
        
    return x_center, y_center, norm_w, norm_h

# Apply the conversion
yolo_coords = []
for index, row in df_bbox.iterrows():
    coords = bbox_to_yolo_format(row['bbox'], row['image_width'], row['image_height'])
    if coords:
        yolo_coords.append(coords)
    else:
        # Append NaNs if conversion failed, to maintain DataFrame structure
        yolo_coords.append((np.nan, np.nan, np.nan, np.nan))


df_bbox[['x_center', 'y_center', 'width_norm', 'height_norm']] = pd.DataFrame(yolo_coords, index=df_bbox.index)

# Drop rows where conversion might have failed (if any)
df_bbox.dropna(subset=['x_center'], inplace=True)

# Assign class_id (all are 'tb' so class_id is 0)
df_bbox['class_id'] = class_id
print(f"📊 Processed {len(df_bbox)} bounding boxes into YOLO format.")

# === 5. Split Data by Image Filename ===
# Get unique filenames to ensure images are not split across train/val
unique_filenames = df_bbox['fname'].unique()
train_fnames, val_fnames = train_test_split(
    unique_filenames,
    test_size=VAL_SPLIT_RATIO,
    random_state=RANDOM_STATE,
    shuffle=True
)

train_df = df_bbox[df_bbox['fname'].isin(train_fnames)].copy()
val_df = df_bbox[df_bbox['fname'].isin(val_fnames)].copy()

print(f"📊 Split: {len(train_fnames)} train images, {len(val_fnames)} validation images.")
print(f"📊 Resulting in: {len(train_df)} train annotations, {len(val_df)} validation annotations.")


# === 6. Process and Save Data for Each Split ===
def process_and_save_split(df_split, image_dest_dir, label_dest_dir, split_name):
    print(f"\n⚙️ Processing {split_name} split...")
    copied_images = set() # To copy each image only once

    for fname, group in df_split.groupby('fname'):
        # --- Copy Image (once per unique filename) ---
        if fname not in copied_images:
            src_img_path = os.path.join(image_source_dir, fname)
            dst_img_path = os.path.join(image_dest_dir, fname)
            if os.path.exists(src_img_path):
                copyfile(src_img_path, dst_img_path)
                copied_images.add(fname)
            else:
                print(f"⚠️ Image missing: {src_img_path}. Skipping this image.")
                continue # Skip label creation if image is missing

        # --- Create Label File (one file per image, multiple lines for multiple bboxes) ---
        label_filename = os.path.splitext(fname)[0] + ".txt"
        label_path = os.path.join(label_dest_dir, label_filename)

        with open(label_path, 'w') as f: # Use 'w' to overwrite if file exists from a previous run
            for _, row in group.iterrows():
                # Format: class_id x_center y_center width height
                yolo_line = f"{int(row['class_id'])} {row['x_center']:.6f} {row['y_center']:.6f} {row['width_norm']:.6f} {row['height_norm']:.6f}\n"
                f.write(yolo_line)
    print(f"✅ Processed {len(copied_images)} images and their labels for {split_name} split.")

process_and_save_split(train_df, train_images_dir, train_labels_dir, "train")
process_and_save_split(val_df, val_images_dir, val_labels_dir, "validation")

# === 7. Create data.yaml file ===
# Paths for data.yaml should be relative to the base_dir if yaml is also in base_dir
# Or absolute if YOLO will be run from a different location.
# For simplicity, using paths relative to where the YAML file will be.
yaml_content = f"""
train: ./images/train
val: ./images/val

nc: 1  # number of classes
names: ['{class_name}'] # class names
"""

yaml_path = os.path.join(base_dir, "data.yaml")
with open(yaml_path, 'w') as f:
    f.write(yaml_content)

print(f"\n📄 data.yaml created at: {yaml_path}")
print("\n🎉 Dataset preparation complete!")
print(f"➡️  Root path: {base_dir}")
print(f"    images/train: {len(os.listdir(train_images_dir))} files")
print(f"    images/val:   {len(os.listdir(val_images_dir))} files")
print(f"    labels/train: {len(os.listdir(train_labels_dir))} files")
print(f"    labels/val:   {len(os.listdir(val_labels_dir))} files")

📁 Created YOLO directory structure in: D:\yolo_training_tb_only
📄 Loaded 8811 rows from D:\yolo_training\data.csv
👍 Filtered to 1211 rows with bounding boxes.
📊 Processed 1211 bounding boxes into YOLO format.
📊 Split: 639 train images, 160 validation images.
📊 Resulting in: 973 train annotations, 238 validation annotations.

⚙️ Processing train split...
✅ Processed 639 images and their labels for train split.

⚙️ Processing validation split...
✅ Processed 160 images and their labels for validation split.

📄 data.yaml created at: D:\yolo_training_tb_only\data.yaml

🎉 Dataset preparation complete!
➡️  Root path: D:\yolo_training_tb_only
    images/train: 639 files
    images/val:   160 files
    labels/train: 639 files
    labels/val:   160 files


In [19]:
import os
import shutil
import json
import time
import torch
import pandas as pd
import matplotlib.pyplot as plt
from ultralytics import YOLO

# === 1. Configuration ===
base_dir = r"D:\yolo_training_tb_only"
data_yaml_path = os.path.join(base_dir, "data.yaml")
model_name = 'yolov8m.pt'  # Using official YOLOv8 medium model
epochs = 100
image_size = 640
patience_epochs = 20
project_name = "yolov8m_tb_training"
experiment_name = "run1"
batch_size = 8
workers = 4
device_to_use = 0  # Set to 0 for GPU, 'cpu' for CPU

# === 2. Sanity Checks ===
if not os.path.exists(data_yaml_path):
    raise FileNotFoundError(f"data.yaml not found at {data_yaml_path}")

# === 3. Initialize and Train YOLOv8m Model ===
print(f"Initializing YOLO model: {model_name}")
model = YOLO(model_name)

start_time = time.time()

print("Starting YOLOv8m training...")
results = model.train(
    data=data_yaml_path,
    epochs=epochs,
    imgsz=image_size,
    batch=batch_size,
    patience=patience_epochs,
    project=os.path.join(base_dir, project_name),
    name=experiment_name,
    device=device_to_use,
    workers=workers,
    exist_ok=True
)

end_time = time.time()
training_duration = end_time - start_time
print(f"Training completed in {training_duration:.2f} seconds.")

# === 4. Locate and Copy Best Weights ===
run_dir = results.save_dir
weights_dir = os.path.join(run_dir, 'weights')
best_weights_path = os.path.join(weights_dir, 'best.pt')
last_weights_path = os.path.join(weights_dir, 'last.pt')

final_best_weight_name = f"best_{os.path.splitext(model_name)[0]}_tb_e{epochs}_b{batch_size}.pt"
destination_best_weights_path = os.path.join(base_dir, final_best_weight_name)

if os.path.exists(best_weights_path):
    shutil.copy2(best_weights_path, destination_best_weights_path)
    print(f"Best weights copied to: {destination_best_weights_path}")
else:
    print("best.pt not found. Checking for last.pt.")
    if os.path.exists(last_weights_path):
        destination_last_weights_path = os.path.join(base_dir, f"last_{os.path.splitext(model_name)[0]}_tb_e{epochs}_b{batch_size}.pt")
        shutil.copy2(last_weights_path, destination_last_weights_path)
        print(f"Last weights copied to: {destination_last_weights_path}")
    else:
        print(f"Neither best.pt nor last.pt found in {weights_dir}.")

# === 5. Performance Metrics ===
results_csv_path = os.path.join(run_dir, 'results.csv')
json_summary_path = os.path.join(run_dir, 'training_summary.json')

if os.path.exists(results_csv_path):
    df_results = pd.read_csv(results_csv_path)
    df_results.columns = df_results.columns.str.strip().str.replace('/', '_', regex=False).str.replace('(', '_', regex=False).str.replace(')', '', regex=False)

    last_epoch_metrics = df_results.iloc[-1].to_dict()
    last_epoch_metrics['training_duration_sec'] = training_duration

    with open(json_summary_path, 'w') as f:
        json.dump(last_epoch_metrics, f, indent=4)
    print(f"Training metrics saved to: {json_summary_path}")

    # === Plotting ===
    plt.figure(figsize=(15, 10))

    # Box Loss
    plt.subplot(2, 2, 1)
    if 'train_box_loss' in df_results.columns and 'val_box_loss' in df_results.columns:
        plt.plot(df_results['epoch'], df_results['train_box_loss'], label='Train Box Loss')
        plt.plot(df_results['epoch'], df_results['val_box_loss'], label='Validation Box Loss')
        plt.title('Box Loss vs. Epochs')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.legend()
    else:
        plt.text(0.5, 0.5, 'Box loss data not found', ha='center', va='center')

    # mAP
    plt.subplot(2, 2, 2)
    if 'metrics_mAP50_B' in df_results.columns:
        plt.plot(df_results['epoch'], df_results['metrics_mAP50_B'], label='mAP@0.50 (B)')
    if 'metrics_mAP50-95_B' in df_results.columns:
        plt.plot(df_results['epoch'], df_results['metrics_mAP50-95_B'], label='mAP@0.50-0.95 (B)')
    if 'metrics_mAP50_B' in df_results.columns or 'metrics_mAP50-95_B' in df_results.columns:
        plt.title('mAP vs. Epochs')
        plt.xlabel('Epochs')
        plt.ylabel('mAP')
        plt.legend()
    else:
        plt.text(0.5, 0.5, 'mAP data not found', ha='center', va='center')

    # Precision & Recall
    plt.subplot(2, 2, 3)
    if 'metrics_precision_B' in df_results.columns:
        plt.plot(df_results['epoch'], df_results['metrics_precision_B'], label='Precision (B)')
    if 'metrics_recall_B' in df_results.columns:
        plt.plot(df_results['epoch'], df_results['metrics_recall_B'], label='Recall (B)')
    if 'metrics_precision_B' in df_results.columns or 'metrics_recall_B' in df_results.columns:
        plt.title('Precision & Recall vs. Epochs')
        plt.xlabel('Epochs')
        plt.ylabel('Score')
        plt.legend()
    else:
        plt.text(0.5, 0.5, 'Precision/Recall data not found', ha='center', va='center')

    # Class Loss
    plt.subplot(2, 2, 4)
    if 'train_cls_loss' in df_results.columns and 'val_cls_loss' in df_results.columns:
        plt.plot(df_results['epoch'], df_results['train_cls_loss'], label='Train Class Loss')
        plt.plot(df_results['epoch'], df_results['val_cls_loss'], label='Validation Class Loss')
        plt.title('Class Loss vs. Epochs')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.legend()
    else:
        plt.text(0.5, 0.5, 'Class loss data not found', ha='center', va='center')

    plt.tight_layout()
    plots_save_path = os.path.join(run_dir, 'performance_plots.png')
    plt.savefig(plots_save_path)
    print(f"Performance plots saved to: {plots_save_path}")
    plt.close()
else:
    print(f"results.csv not found at {results_csv_path}. Cannot display detailed metrics or plots.")

# === 6. GPU Memory Usage ===
if torch.cuda.is_available():
    gpu_memory_allocated = torch.cuda.memory_allocated(device_to_use) / (1024 ** 3)
    gpu_memory_reserved = torch.cuda.memory_reserved(device_to_use) / (1024 ** 3)
    print(f"GPU Memory Allocated: {gpu_memory_allocated:.2f} GB")
    print(f"GPU Memory Reserved: {gpu_memory_reserved:.2f} GB")
else:
    print("CUDA is not available. Skipping GPU memory usage logging.")

# === 7. Display Confusion Matrix ===
confusion_matrix_path = os.path.join(run_dir, 'confusion_matrix.png')
confusion_matrix_normalized_path = os.path.join(run_dir, 'confusion_matrix_normalized.png')

if os.path.exists(confusion_matrix_path):
    print(f"Confusion Matrix saved at: {confusion_matrix_path}")
    if os.path.exists(confusion_matrix_normalized_path):
        print(f"Normalized Confusion Matrix saved at: {confusion_matrix_normalized_path}")
else:
    print("Confusion matrix plot not found.")


Initializing YOLO model: yolov8m.pt


100%|██████████| 49.7M/49.7M [01:37<00:00, 533kB/s] 


Starting YOLOv8m training...
Ultralytics 8.3.149  Python-3.13.3 torch-2.7.1+cu128 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=D:\yolo_training_tb_only\data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8m.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=run1, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=20

train: Scanning D:\yolo_training_tb_only\labels\train.cache... 639 images, 0 backgrounds, 0 corrupt: 100%|██████████| 639/639 [00:00<?, ?it/s]


val: Fast image access  (ping: 0.10.0 ms, read: 5.72.6 MB/s, size: 368.7 KB)


...

     35/100      3.42G      1.683      1.585      1.796         14        640: 100%|██████████| 80/80 [01:09<00:00,  1.16it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:25<00:00,  2.50s/it]

                   all        160        238      0.736      0.639       0.72      0.318



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     36/100      3.44G      1.691      1.591      1.823         14        640: 100%|██████████| 80/80 [00:42<00:00,  1.89it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  60%|██████    | 6/10 [00:09<00:06,  1.53s/it]


KeyboardInterrupt: 